In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.20.0


In [ ]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 32

# Paths (Adjust if your notebook is in a different subdirectory)
BASE_DIR = "../dataset" 
TRAIN_DIR = os.path.join(BASE_DIR, "train")
TEST_DIR = os.path.join(BASE_DIR, "test")

print(f"Checking if paths exist: {os.path.exists(TRAIN_DIR)}")

Checking if paths exist: True


In [ ]:
# 1. Instantiate the Generator (Normalizing pixel values from 0-255 to 0-1)
datagen = ImageDataGenerator(rescale=1./255)

# 2. Flow images from the directory structures
print("Loading Training Data:")
train_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical', # Used for multi-class classification
    shuffle=True
)

# Check the shape of the images in a batch
X_batch, y_batch = next(train_generator)  # Get a batch of images and labels
print("Batch image shape:", X_batch.shape)  # Should print (BATCH_SIZE, 224, 224, 3)

print("\nLoading Test Data:")
test_generator = datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False # Usually False for testing
)
X_batch, y_batch = next(test_generator)  # Get a batch of images and labels
print("Batch image shape:", X_batch.shape)  # Should print (BATCH_SIZE, 224, 224, 3)


Loading Training Data:
Found 2870 images belonging to 4 classes.
Batch image shape: (32, 224, 224, 3)

Loading Test Data:
Found 394 images belonging to 4 classes.


In [ ]:
# Map integer indices to human-readable names
labels = {v: k for k, v in train_generator.class_indices.items()}
print("\nClass Map:", labels)


Class Map: {0: 'glioma_tumor', 1: 'meningioma_tumor', 2: 'no_tumor', 3: 'pituitary_tumor'}


In [1]:
# Load data generators
train_generator, test_generator = create_data_generators(TRAIN_DIR, TEST_DIR, IMG_SIZE, BATCH_SIZE)

# Create a class map
labels = get_class_map(train_generator)
print("\nClass Map:", labels)

illegal character: '#'
# Load data generators
^
';' expected
# Load data generators
                      ^
